In [58]:
import pandas as pd
import os
from sqlalchemy import create_engine, text

# Setup paths
path = "./materials/"
os.makedirs(path, exist_ok=True)

xlsx_file = os.path.join(path, "materials.xlsx")
db_file = os.path.join(path, "materials.db")
engine = create_engine(f"sqlite:///{os.path.abspath(db_file)}")

# Load data
df = pd.read_excel(xlsx_file)

# Use transaction block that auto-commits
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS materials"))

    conn.execute(text("""
        CREATE TABLE materials (
            id INTEGER PRIMARY KEY,
            name TEXT,
            grade TEXT,
            chemical_formula TEXT,
            composition_by_weight_percent TEXT,
            mass_density_g_cm3 REAL,
            data_source TEXT
        )
    """))

    for _, row in df.iterrows():
        conn.execute(text("""
            INSERT INTO materials (id, name, grade, chemical_formula,
                                   composition_by_weight_percent,
                                   mass_density_g_cm3, data_source)
            VALUES (:id, :name, :grade, :chemical_formula, :composition,
                    :density, :source)
        """), {
            "id": int(row["id"]),
            "name": row["name"],
            "grade": row["grade"],
            "chemical_formula": row["chemical_formula"],
            "composition": row["composition_by_weight_percent"],
            "density": row["mass_density_g_cm3"],
            "source": row["data_source"]
        })

print(f"✅ materials.db committed with {len(df)} materials.")


✅ materials.db committed with 16 materials.


In [59]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT id, name FROM materials"))
    for row in result:
        print(f"{row.id}: {row.name}")

1: Al
2: BeCu
3: TiAlV
4: TiZr
5: SS304
6: SS316
7: V
8: VNb
9: NiCrAl
10: Teflon
11: Re
12: W
13: WC
14: sinteredDiamond
15: versimax
16: ZTA


In [54]:
print("Using database at:", os.path.abspath("./materials/materials.db"))

Using database at: /Users/66j/Documents/ORNL/code/SEEMeta/materials/materials.db


In [55]:
engine = create_engine("sqlite:///materials/materials.db")  # match exactly

In [56]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table'"))
    print("Tables in database:")
    for row in result:
        print("-", row.name)

Tables in database:
- sqlite_sequence
- spectra
- materials


In [57]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT id, name FROM materials")).fetchall()
    print(f"Found {len(result)} materials:")
    for row in result:
        print(f"- {row.id}: {row.name}")

Found 0 materials:
